In [ ]:
import pandas as pd
import glob
import numpy as np

In [ ]:
df = pd.read_parquet('../data/all_data.parquet')

In [ ]:
df.to_csv('../data/comics_data.csv')

In [ ]:
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 500
pd.options.display.max_columns = 100

In [ ]:
# combine all partial dataframes


path = r'./single_issue_data' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True).reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
df.to_csv('single_issues_data.csv')

In [ ]:
df.head()

In [ ]:
df_issue_list = pd.read_csv('all_issues_data.csv')

In [ ]:
df_issue_list.head(3)

In [ ]:
df_issue_list.shape

In [ ]:
df_issue_list.info()

In [ ]:
df_pub = pd.read_csv('publishers_data.csv', index_col=0)

In [ ]:
df_pub.head(3)

In [ ]:
df_title = pd.read_csv('titles_list_data.csv', index_col=0)

In [ ]:
df_title.shape

In [ ]:
df_title.head()

In [ ]:
df

In [ ]:
issue_list_to_merge = df_issue_list[['pub_id', 'issue_link']]

In [ ]:
issue_list_to_merge.issue_link = issue_list_to_merge.issue_link.apply(lambda s: 'https://comicbookrealm.com' + s)

In [ ]:
issue_list_to_merge

In [ ]:
pubs_to_merge = df_pub[['id', 'name', 'titles_tracking', 'comics_tracking']].rename({'id': 'pub_id'}, axis=1)

In [ ]:
title_to_merge = df_title[['title_link', 'volume', 'years', 'issues']]

In [ ]:
dfm = df.merge(issue_list_to_merge, on='issue_link', how='left')

In [ ]:
dfm.info()

In [ ]:
dfm1 = dfm.merge(title_to_merge, on='title_link', how='left').rename({'issues': 'issues_total'}, axis=1)

In [ ]:
dfm1.shape

In [ ]:
dfm2 = dfm1.merge(pubs_to_merge, on='pub_id', how='left')

In [ ]:
dfm2

In [ ]:
dff = dfm2.rename({'name': 'pub_name', 'titles_tracking': 'pub_titles_total', 
                   'comics_tracking': 'pub_issues_total'}, axis=1)

In [ ]:
dff.info(memory_usage='deep')

In [ ]:
dff.pages = pd.to_numeric(dff.pages, errors='coerce')

In [ ]:
dff.issues_total = pd.to_numeric(dff.issues_total.str.replace(',', ''))

In [ ]:
dff.pub_titles_total = pd.to_numeric(dff.pub_titles_total.str.replace(',', ''))

In [ ]:
dff.pub_issues_total = pd.to_numeric(dff.pub_issues_total.str.replace(',', ''))

In [ ]:
dff.to_parquet('all_data.parquet', engine='pyarrow')